In [46]:
%load_ext autoreload
%autoreload 2
import manenv
from manenv.utils.product_utils import *
from manenv.core.world import World
from manenv.core.component import *
from manenv.render_wrapper import RenderWrapper
from manenv.core.product import *
from manenv.core.effector import *

import manenv.effectors as eff
import manenv.components as cmp


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
world = World((3, 3))
s = np.zeros((5, 5), dtype=np.int8)
s[1][2] = 2
s[1][1] = 2
s[2][1] = 1
s[3][1] = 1
product = Product(s)
world.place_component(make_vector(1, 1), cmp.Conveyor(
    np.array([
    [0, 0, 0],
    [1, 0, -1],
    [0, 0, 0]
])))
world.place_component(make_vector(2, 0), cmp.Conveyor(
    np.array([
    [0, -1, 0],
    [0, 0, 1],
    [0, 0, 0] 
])))
world.place_component(make_vector(2, 1), cmp.Outport())

grabber = eff.Grabber(make_vector(0, 2))
buddy = eff.Grabber(make_vector(1, 3))
welder = eff.Welder(make_vector(1, 2))
transformer = eff.Transformer(1, 2, make_vector(2,  4))
discard = eff.Acceptor(make_vector(1, 7))

world.place_component(make_vector(1, 2), cmp.Spawner(product))
assembler : cmp.Assembler = cmp.Assembler(make_vector(10, 10), [grabber, buddy, welder, discard])
world.place_component(make_vector(1, 0), assembler)
wrapper = RenderWrapper(world)

In [48]:
# wrapper.reset()
# assembler.place_in_inventory(product.copy())
# assembler.place_in_inventory(product.copy())
# world.update()

# grabber.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
# grabber.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
# buddy.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
# world.update()

# grabber.set_action(eff.GrabberActions.RELEASE.value)
# buddy.set_action(eff.GrabberActions.RELEASE.value)
# world.update()

# welder.set_action(eff.WelderActions.WELD_EAST.value)
# world.update()
# world.update()

# grabber.set_action(eff.GrabberActions.GRAB.value)
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
# world.update()

# grabber.set_action(eff.GrabberActions.RELEASE.value)
# world.update()

# transformer.set_action(eff.TransformerActions.TRANSFORM.value)
# world.update()
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
# world.update()
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
# world.update()


In [49]:
# wrapper.render()

# Gym Environment Test

In [50]:
from manenv.gym_wrapper import MARLFactoryEnvironment

In [51]:
world.reset()
environment = MARLFactoryEnvironment(world)
renderer = RenderWrapper(environment._world)

In [52]:
from pettingzoo.test import parallel_api_test

In [53]:
def test():
    parallel_api_test(environment, num_cycles=1_000_000)

In [54]:
import cProfile
cProfile.run('test()', sort = 'time')

{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 0}
{72: 1}
{72: 2}
{72: 3}
{72: 4}
{72: 5}
{72: 6}
{72: 7}
{72: 8}
{72: 9}
{72: 10}
{72: 11}
{72: 12}
{72: 13}
{72: 14}
{72: 15}
{72: 16}
{72: 17}
{72: 18}
{72: 19}
{72: 20}
{72: 21}
{72: 22}
{72: 23}
{72: 24}
{72: 25}
{72: 26}
{72: 27}
{72: 28}
{72: 29}
{72: 30}
{72: 31}
{72: 32}
{72: 33}
{72: 34}
{72: 35}
{72: 36}
{72: 37}
{72: 38}
{72: 39}
{72: 40}
{72: 41}
{72: 42}
{72: 43}
{72: 44}
{72: 45}
{72: 46}
{72: 47}
{72: 48}
{72: 49}
{72: 50}
{72: 51}
{72: 52}
{72: 53}
{72: 54}
{72: 55}
{72: 56}
{72: 57}
{72: 58}
{72: 59}
{72: 60}
{72: 61}
{72: 62}
{72: 63}
{72: 64}
{72: 65}
{72: 66}
{72: 67}
{72: 68}
{72: 69}
{72: 70}
{72

c:\Users\Joaquin\miniconda3\envs\thesis\Lib\site-packages\pettingzoo\test\parallel_test.py:88: UserWarning: Live agent was not given reward
  warnings.warn(f"Live agent was not given {k}")


{72: 181}
{72: 182}
{72: 183}
{72: 184}
{72: 185}
{72: 186}
{72: 187}
{72: 188}
{72: 189}
{72: 190}
{72: 191}
{72: 192}
{72: 193}
{72: 194}
{72: 195}
{72: 196}
{72: 197}
{72: 198}
{72: 199}
{72: 200}
{72: 201}
{72: 202}
{72: 203}
{72: 204}
{72: 205}
{72: 206}
{72: 207}
{72: 208}
{72: 209}
{72: 210}
{72: 211}
{72: 212}
{72: 213}
{72: 214}
{72: 215}
{72: 216}
{72: 217}
{72: 218}
{72: 219}
{72: 220}
{72: 221}
{72: 222}
{72: 223}
{72: 224}
{72: 225}
{72: 226}
{72: 227}
{72: 228}
{72: 229}
{72: 230}
{72: 231}
{72: 232}
{72: 233}
{72: 234}
{72: 235}
{72: 236}
{72: 237}
{72: 238}
{72: 239}
{72: 240}
{72: 241}
{72: 242}
{72: 243}
{72: 244}
{72: 245}
{72: 246}
{72: 247}
{72: 248}
{72: 249}
{72: 250}
{72: 251}
{72: 252}
{72: 253}
{72: 254}
{72: 255}
{72: 256}
{72: 257}
{72: 258}
{72: 259}
{72: 260}
{72: 261}
{72: 262}
{72: 263}
{72: 264}
{72: 265}
{72: 266}
{72: 267}
{72: 268}
{72: 269}
{72: 270}
{72: 271}
{72: 272}
{72: 273}
{72: 274}
{72: 275}
{72: 276}
{72: 277}
{72: 278}
{72: 279}
{72: 280}


: 

In [ ]:
for _ in range(100):
    action = environment._action_space.sample()
    environment.step(action)
renderer.render()

KeyError: 12